In [4]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [92]:
batch_size = 128
num_classes = 10
epochs = 50

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
r = 25

In [93]:
x_train.shape

(60000, 28, 28)

In [94]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [106]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [107]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(28000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)


In [108]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (28000, 28, 28, 1)
y_train shape: (60000, 10)
28000 train samples
10000 test samples


In [98]:
y_train.shape

(60000, 10)

In [99]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
__________

In [100]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [101]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 64s 1ms/step - loss: 0.2697 - acc: 0.9171 - val_loss: 0.0555 - val_acc: 0.9821
Epoch 2/50
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0905 - acc: 0.9725 - val_loss: 0.0425 - val_acc: 0.9858
Epoch 3/50
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0648 - acc: 0.9806 - val_loss: 0.0344 - val_acc: 0.9874
Epoch 4/50
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0548 - acc: 0.9836 - val_loss: 0.0326 - val_acc: 0.9886
Epoch 5/50
60000/60000 [==============================] - 60s 1ms/step - loss: 0.0457 - acc: 0.9863 - val_loss: 0.0306 - val_acc: 0.9894
Epoch 6/50
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0418 - acc: 0.9875 - val_loss: 0.0316 - val_acc: 0.9894
Epoch 7/50
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0369 - acc: 0.9884 - val_loss: 0.0276 - val_acc

In [102]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.028092098616300246
Test accuracy: 0.9936


In [103]:
dt=pd.read_csv(r'D:\ML\DATA\Digit Recognizer\test.csv')

In [104]:
dt.head(1)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
x_train=dt.values

In [45]:
x_train.shape

(28000, 784)

In [109]:
import numpy as np
# predict results
results = model.predict(x_train)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [111]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen_mine.csv",index=False)